In [ ]:
# DS-22 Самощенков Сергей
# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [104]:

# Задание 1.
# Обязательная часть
# Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
# Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
# KEYWORDS = ['python', 'парсинг']
# Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
# В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
# Дополнительная часть (необязательная)
# Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
# Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
# Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

import requests
import time 
import pandas as pd
from bs4 import BeautifulSoup

# определяем список хабов, которые нам интересны
keywords = ['python', 'парсинг']
hub_news = pd.DataFrame()

# получаем страницу с самыми свежими постами
req = requests.get('http://habr.com/ru/all/')
soap = BeautifulSoup(req.text, 'html.parser')

# извлекаем посты
posts = soup.find_all('article', class_ ='post')

# Вариант 1

hubs = list(map(lambda x: x.find('a', class_='post__title_link'), posts))
hub_links = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), posts))
hub_titles = list(map(lambda x: x.find('a', class_='post__title_link').getText(), posts))
hub_dates = list(map(lambda x: x.find('span', class_='post__time').getText(), posts))
hub_authors = list(map(lambda x: x.find('span', class_='user-info__nickname').getText(), posts))
hub_ids = list(map(lambda x: x.parent.attrs.get('id'), posts))

if len(hub_links) == len(hub_titles) == len(hub_dates) == len(hub_authors) == len(hub_ids):
    hub_news['hub_links']=hub_links
    hub_news['hub_titles']=hub_titles
    hub_news['hub_dates']=hub_dates
    hub_news['hub_authors']=hub_authors
    hub_news['hub_ids']=hub_ids
    hub_news = hub_news[hub_news['hub_titles'].apply(lambda x: any([keyword in x.lower() for keyword in keywords]))]
else:
    print ('Данный алгоритм не подходит')

# Вариант 2
hub_news2 = pd.DataFrame()

for index, post in enumerate(posts):
    # достаем id статьи
    hub_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
    if not hub_id:
        continue
    # вычленяем числовую часть номера: делим 'post_127464' на части по разделителю '_' и забираем последнюю часть
    hub_id = int(hub_id.split('_')[-1]) 
    # достаем другие параметры статьи
    hub_link = post.find('a', class_='post__title_link').get('href')
    hub_title = post.find('a', class_='post__title_link').getText()
    hub_date = post.find('span', class_='post__time').getText()
    hub_author = post.find('span', class_='user-info__nickname').getText()
    hub_text = ''
    # проверяем заголовок на соответствие условиям поиска
    if any([keyword in hub_title.lower() for keyword in keywords]):
        # достаем полный текст статьи
        soup = BeautifulSoup(requests.get(hub_link).text, 'html.parser')
        time.sleep(0.2)
        if soap.find('div', class_='post__text post__text-html post__text_v1'):
            hub_text = soap.find('div', class_='post__text post__text-html post__text_v1').text
        # создаем итоговый датафрейм
        row = {'hub_title': hub_title, 'hub_date': hub_date, 'hub_author': hub_author, 'hub_link': hub_link, 'hub_id': hub_id, 'hub_text': hub_text}
        hub_news2 = hub_news2.append(pd.DataFrame([row]))    

hub_news2




""


In [155]:
# Задание 2. - 

# Обязательная часть
# Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

# mail = {'emailAddresses': ["saswin18178.coolteam@gmail.com"]}
# В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

# Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

import requests
import time 
import pandas as pd
from bs4 import BeautifulSoup

EMAIL = ['saswin18178.coolteam@gmail.com', 'saswin@yandex.ru']

req_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '45',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'sec-ch-ua-mobile': '?0',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0',
}

def find_breaches(mail):
    """
    Возвращает датафрейм со списком взломов для заданного ящика
    Поля датафрейма: 'mail', 'status' = hacked|not hacked, 'date', 'site', 'description'
    """
    time.sleep(0.3)
    json_mail = {'emailAddresses': [mail]}
    req = requests.post(req_url, json = json_mail, headers = headers)    
    req_df = pd.DataFrame(req.json())

    breaches_count = len(req_df.index)//2
    mail_breaches_df = pd.DataFrame(columns=['mail', 'status', 'date', 'site', 'description'])  
    
    if breaches_count == 0:
        row = {'mail': mail, 'status': 'not hacked', 'date' : None, 'site' : None, 'description' : None}
        mail_breaches_df = mail_breaches_df.append(pd.DataFrame([row]))  
    else: 
        for i in range(breaches_count):
            br = req_df.iloc[i]['breaches']
            publishDate = br['publishDate']
            site = br['site']
            description = br['description']
            row = {'mail': mail, 'status': 'hacked', 'date' : publishDate, 'site' : site, 'description' : description}
            mail_breaches_df = mail_breaches_df.append(pd.DataFrame([row]))
    
    return mail_breaches_df

breaches_df = pd.DataFrame(columns=['mail', 'status', 'date', 'site', 'description'])

for mail in EMAIL:
    mail_breaches_df = find_breaches(mail)
    breaches_df = pd.concat([breaches_df, mail_breaches_df], ignore_index=True)

breaches_df.head(35)



,mail,status,date,site,description
0,saswin18178.coolteam@gmail.com,not hacked,None,None,None
1,saswin@yandex.ru,hacked,2016-10-25T00:00:00Z,tumblr.com,"In May 2016, microblogging platform Tumblr rev..."
2,saswin@yandex.ru,hacked,2018-10-04T00:00:00Z,autowebinar.fm,"In December 2013, AutoWebinar FM's database wa..."
3,saswin@yandex.ru,hacked,2020-12-10T00:00:00Z,gdepoigrat.ru,"In November 2020, a collection of over 23,000 ..."


In [ ]:
# Дополнительная часть (необязательная)
# Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
# Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

# GROUP = 'netology'  
# TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
# В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>


# params = {
#     'target_type': 'posts',
#     'q': {keywords[0]},
#     'order_by': 'date'
# }
# URL = 'https://habr.com/ru/search/'
# headers = {}

# time.sleep(0.2)

# req = requests.get(URL, params)
# req = requests.post(URL, params, headers)
# req.text

# soup = BeautifulSoup(req.text, 'html.parser')
# posts = soup.find_all('article', class_='post')

# for post in posts:
#     post_id = post.parent.attrs.get('id')
#     if not post_id:
#         continue
#     post_id = int(post_id)